In [17]:
from src.retrieval import PyLuceneRetriever, ReCodeRetriever, OracleBLEURetriever
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from src.BERT_style_modules import BERTStyleEncoder, BertTokenizer

from src.useful_utils import string_split_v3, string_split_v2, string_split_v1
from src.metrics import nltk_bleu
from src.dataset_loaders import SRC_TGT_pairs
from src.vocab_classes import Shared_Vocab
import numpy as np
import tqdm.notebook as tqdm 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
num_folds = 10

In [3]:
def fold_iterator(file_format="datasets/django_folds/django.fold<FOLD>-10.<MODE>.<EXT>", folds, train_mode="train", test_mode="test", src="src", tgt="tgt"):
    for fold_idx in range(folds):
        src_train_fp = file_format.replace()
        tgt_train_fp
        src_test_fp
        tgt_test_fp
        test_samples = SRC_TGT_pairs(args.src_test_fp, args.tgt_test_fp).samples

SyntaxError: non-default argument follows default argument (<ipython-input-3-e7b695c25420>, line 1)

In [4]:
nltk_bleu(["foo","bar"], ["foo","baz"])

0.7071067811865476

In [17]:
nltk_bleu("foo bar", "foo baz")

0.8091067115702212

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer.encode("raise an exception of class ValueError, with string")[1:-1]

[5333, 2019, 6453, 1997, 2465, 3643, 2121, 29165, 1010, 2007, 5164]

In [30]:
a = time.time()

In [40]:
import datetime

b = time.time() - a
b = b*800
str(datetime.timedelta(seconds=b))

'2 days, 2:35:29.742432'

In [24]:
retriever = PyLuceneRetriever()
retriever.add_multiple_docs(["potatoes come from ireland", "i love potatoes", "starch comes from potatoes", "there is starch in ireland"])
retriever.search("starch^0 ireland^1")

JVM Running


[(0, 0.3150669038295746), (3, 0.2858338952064514), (2, 0.0)]

In [48]:
def retriever_eval(src_train_fp, tgt_train_fp, src_test_fp, tgt_test_fp, k1=1.2, b=0.75):
    train_samples = SRC_TGT_pairs(src_train_fp, tgt_train_fp,max_seq_len=1000).samples
    test_samples = SRC_TGT_pairs(src_test_fp, tgt_test_fp,max_seq_len=1000).samples
    
#     retriever = PyLuceneRetriever()
    retriever = OracleBLEURetriever()

    src_train_data = [string_split_v3(src) for src,tgt in train_samples]
    tgt_train_data = [tgt for src,tgt in train_samples]
    retriever.add_multiple_docs(train_samples)
    
    custom_BLEUs = []
#     BERT_BLEUs = []
#     char_BLEUs = []
    for src, tgt in tqdm.tqdm(test_samples):
#         doc_ranking = retriever.BM25_search(src, k1=k1, b=b)
        doc_ranking = retriever.search(src, tgt)
#         doc_ids = [x.doc for x in doc_ranking]
        doc_ids = doc_ranking
        retrieved_samples = [tgt_train_data[i] for i, s in doc_ids]
        
#         top_code = tgt_train_data[doc_ranking[0].doc] if len(doc_ranking)>0 else ""
        top_code = tgt_train_data[doc_ranking[0][0]] if len(doc_ranking)>0 else ""

        custom_BLEU = nltk_bleu(string_split_v3(top_code), string_split_v3(tgt))
#         BERT_BLEU = nltk_bleu(tokenizer.encode(top_code)[1:-1], tokenizer.encode(tgt)[1:-1])
#         char_BLEU = nltk_bleu(top_code, tgt)
        
        custom_BLEUs.append(custom_BLEU)
#         BERT_BLEUs.append(BERT_BLEU)
#         char_BLEUs.append(char_BLEU)
        
#         print(tgt, "||||||", top_code, "|||| BLEU", BLEU)
    return custom_BLEUs
    
    
# src_train_fp = "datasets/merged/merged_Dj_HS_Co.train.src"
# tgt_train_fp = "datasets/merged/merged_Dj_HS_Co.train.tgt"
# src_test_fp = "datasets/HS/test_hs.in"
# tgt_test_fp = "datasets/HS/test_hs.out"

src_train_fp = "datasets/django_folds/django.fold1-10.full_train.src"
tgt_train_fp = "datasets/django_folds/django.fold1-10.full_train.tgt"
src_test_fp = "datasets/django_folds/django.fold1-10.test.src"
tgt_test_fp = "datasets/django_folds/django.fold1-10.test.tgt"

# src_train_fp = "datasets/CoNaLa/conala-train.src"
# tgt_train_fp = "datasets/CoNaLa/conala-train.tgt"
# src_test_fp = "datasets/CoNaLa/conala-test.src"
# tgt_test_fp = "datasets/CoNaLa/conala-test.tgt"

custom_BLEUs = retriever_eval(src_train_fp, tgt_train_fp, src_test_fp, tgt_test_fp, k1=3.27, b=1.0)

print("custom_BLEUs", np.average(custom_BLEUs))
print("BERT_BLEUs", np.average(BERT_BLEUs))
print("char_BLEUs", np.average(char_BLEUs))


custom_BLEUs 0.5449556561713716


NameError: name 'BERT_BLEUs' is not defined

In [30]:
src_train_fp = "datasets/django_folds/django.fold1-10.train.src"
tgt_train_fp = "datasets/django_folds/django.fold1-10.train.tgt"
src_test_fp = "datasets/django_folds/django.fold1-10.test.src"
tgt_test_fp = "datasets/django_folds/django.fold1-10.test.tgt"

train_samples = SRC_TGT_pairs(src_train_fp, tgt_train_fp,max_seq_len=50).samples
test_samples = SRC_TGT_pairs(src_test_fp, tgt_test_fp,max_seq_len=50).samples

src_train_data = [src for src,tgt in train_samples]
tgt_train_data = [tgt for src,tgt in train_samples]


retriever = PyLuceneRetriever()
retriever.add_multiple_docs(src_train_data)

src, tgt = test_samples[200]

src = "return . url encode s ."
tgt = "return self . s"

print("query:",src)
print("code :", tgt)

results = retriever.search(src)
print(results)
for i, score in results:
    print(train_samples[i], score)

JVM Running
query: return . url encode s .
code : return self . s
[(13309, 6.537010669708252), (10445, 5.287431716918945), (3308, 4.803297996520996)]
('return six.text_type(s).encode(encoding, errors).', 'return six . text_type ( s ) . encode ( encoding , errors )') 6.537010669708252
('return url.', 'return url') 5.287431716918945
('raise an CommandError exception with an argument string "couldn\'t download URL %s to %s: %s", where \'%s\' is replaced with url,', 'raise CommandError ( "couldn\'t download URL %s to %s: %s" %  ( url , filename , e ) )') 4.803297996520996


Django from train merged = 43.5
HS from train merged = 60.6
CoNaLa from train merged = 13.6
AVG = 

In [45]:
fold_BLEUs = []
for i in tqdm.tqdm(range(1,11)):
    src_train_fp = "datasets/django_folds/django.fold<NBR>-10.train.src".replace("<NBR>", str(i))
    tgt_train_fp = "datasets/django_folds/django.fold<NBR>-10.train.tgt".replace("<NBR>", str(i))
    src_test_fp = "datasets/django_folds/django.fold<NBR>-10.test.src".replace("<NBR>", str(i))
    tgt_test_fp = "datasets/django_folds/django.fold<NBR>-10.test.tgt".replace("<NBR>", str(i))
    
    fold_BLEUs = fold_BLEUs + retriever_eval(src_train_fp, tgt_train_fp, src_test_fp, tgt_test_fp, k1=4.03, b=0.99)
np.average(fold_BLEUs)

JVM Running
JVM Running
JVM Running
JVM Running
JVM Running
JVM Running
JVM Running
JVM Running
JVM Running
JVM Running



0.4306389213076508

## hyperparam search

In [104]:
rand_iters = 30

# src_train_fp = "datasets/django_folds/django.fold1-10.train.src"
# tgt_train_fp = "datasets/django_folds/django.fold1-10.train.tgt"
# src_test_fp = "datasets/django_folds/django.fold1-10.test.src"
# tgt_test_fp = "datasets/django_folds/django.fold1-10.test.tgt"

# src_train_fp = "datasets/CoNaLa/conala-train.src"
# tgt_train_fp = "datasets/CoNaLa/conala-train.tgt"
# src_test_fp = "datasets/CoNaLa/conala-test.src"
# tgt_test_fp = "datasets/CoNaLa/conala-test.tgt"

src_train_fp = "datasets/merged/merged_Dj_HS_Co.train.src"
tgt_train_fp = "datasets/merged/merged_Dj_HS_Co.train.tgt"
src_test_fp = "datasets/django_folds/django.fold1-10.test.src"
tgt_test_fp = "datasets/django_folds/django.fold1-10.test.tgt"

for i in range(rand_iters):
    k1 = np.random.uniform(0,5)
    b = np.random.uniform(0,1)
    scores = retriever_eval(src_train_fp, tgt_train_fp, src_test_fp, tgt_test_fp, k1=k1, b=b)
    BLEU_score = np.average(scores)
    print(f"k1={k1:.2f} | b={b:.2f} | BLEU={BLEU_score:.3f}")

JVM Running
k1=4.42 | b=0.68 | BLEU=0.407
JVM Running
k1=0.55 | b=0.21 | BLEU=0.345


KeyboardInterrupt: 

In [ ]:
from joblib import Parallel, delayed
import multiprocessing

In [65]:
def BLEU_oracle(src_train_fp, tgt_train_fp, src_test_fp, tgt_test_fp):
    train_samples = SRC_TGT_pairs(src_train_fp, tgt_train_fp).samples
    test_samples = SRC_TGT_pairs(src_test_fp, tgt_test_fp).samples[:100]
    
    num_cores = multiprocessing.cpu_count()
    inputs = tqdm.tqdm([tgt for src, tgt in test_samples])
    
    
    BLEUs = []
#     for test_src, test_tgt in tqdm.tqdm(test_samples):
    def calc_bleu(test_tgt):
        best_BLEUs = []
        for train_src, train_tgt in train_samples:
            BLEU = nltk_bleu(string_split_v3(train_tgt), string_split_v3(test_tgt))
            best_BLEUs.append(BLEU)
#         BLEUs.append(max(best_BLEUs))
        return max(best_BLEUs)
    
    processed_list = Parallel(n_jobs=num_cores)(delayed(calc_bleu)(i) for i in inputs)

    return processed_list

scores = BLEU_oracle(src_train_fp, tgt_train_fp, src_test_fp, tgt_test_fp)

In [66]:
scores

[0.21364350319811704,
 0.7071067811865476,
 1.0,
 0.537284965911771,
 0.24808415001701817,
 0.5266403878479265,
 0.537284965911771,
 0.7506238537503395,
 0.39763536438352537,
 0.7071067811865476,
 0.30739407647563216,
 0.39763536438352537,
 1.0,
 0.537284965911771,
 0.24808415001701817,
 0.6147881529512643,
 0.537284965911771,
 0.7506238537503395,
 0.39763536438352537,
 0.7071067811865476,
 0.30739407647563216,
 0.35930411196308426,
 0.35930411196308426,
 0.30739407647563216,
 0.35930411196308426,
 0.37991784282579627,
 0.4518010018049224,
 0.41113361690051975,
 0.42728700639623407,
 1.0,
 0.41113361690051975,
 0.39763536438352537,
 1.0,
 0.668740304976422,
 1.0,
 0.41113361690051975,
 0.5859059370151705,
 1.0,
 0.30739407647563216,
 1.0,
 0.3972418603247485,
 0.3303164318013807,
 1.0,
 0.33277145517762347,
 0.4347208719449914,
 0.23793665482062612,
 0.3551292228499906,
 0.3945881255591767,
 0.18216041731535024,
 0.20164945583740662,
 1.0,
 0.5081327481546147,
 0.21364350319811704,
 0.

In [67]:
np.average(scores)

0.48330251599852075

In [42]:
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

myList = [1,2,3,4]

def myfunction(val, c):
    return val*3 + c + a

num_cores = multiprocessing.cpu_count()
inputs = tqdm(myList)

parameters = [100, 5]

processed_list = Parallel(n_jobs=num_cores)(delayed(myfunction)(i,*parameters) for i in inputs)
processed_list




100%|██████████| 4/4 [00:00<00:00, 4841.91it/s]


TypeError: myfunction() takes 2 positional arguments but 3 were given